In [1]:
from modules.tweet_data import read_raw_data
from modules.topics import TopicSeries
#from modules.tokens import spacy_twitter_model, twitter_tokenizer

import pandas as pd
import spacy
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

import matplotlib.pyplot as plt

In [2]:
tweet_df = read_raw_data()

Reading data


In [3]:
tweet_df.tail()

,tweet_id,handle,tweet
timestamp,,,
2020-07-11 00:32:55,1281687871406186497,RANsquawk,Cheers good sir!!! you too mate :)
2020-07-11 00:37:10,1281688941389905921,lindayueh,"Prime Minister, who posed in a face covering f..."
2020-07-11 00:52:55,1281692904071876611,bopinion,"Today, roughly 20% of all U.S.-listed stocks a..."
2020-07-11 01:22:53,1281700446281838605,bopinion,We'll have to re-think some of our notions abo...
2020-07-11 01:53:32,1281708159606894598,bopinion,Something important is afoot. Chinese stock ...


In [4]:
tweet_df = tweet_df[tweet_df.index < '06-01-2020']

In [5]:
single_day = tweet_df['2016-11-09']

In [6]:
nlp = spacy_twitter_model()


In [7]:
cv = CountVectorizer(tokenizer=lambda text: twitter_tokenizer(text, model=nlp))
count_vecs = cv.fit_transform(single_day.tweet)

lda = LatentDirichletAllocation(n_components=5, random_state=42)
lda_vecs = lda.fit_transform(count_vecs)

In [9]:
tfidf = TfidfVectorizer(tokenizer=lambda text: twitter_tokenizer(text, model=nlp))
tfidf_vecs = tfidf.fit_transform(single_day.tweet)

nmf = NMF(n_components=5, random_state=42)
nmf_vecs = nmf.fit_transform(tfidf_vecs)

In [3]:
ts =TopicSeries()
ts.calculate_day('2016-11-09', tweet_df['2016-11-09'].tweet)
ts.calculate_day('2016-05-08', tweet_df['2016-05-08'].tweet)

In [25]:
ts.models['2016-11-09']

[LatentDirichletAllocation(n_components=5, random_state=42),
 CountVectorizer(tokenizer=<function TopicSeries.twitter_tokenizer at 0x7fc5ab2cb320>),
 NMF(n_components=5, random_state=5),
 TfidfVectorizer(tokenizer=<function TopicSeries.twitter_tokenizer at 0x7fc5ab2cb320>)]

In [10]:
def display_components(model, word_features, top_display=5):
    # utility for displaying respresentative words per component for topic models
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        top_words_idx = topic.argsort()[::-1][:top_display]
        top_words = [word_features[i] for i in top_words_idx]
        print(" ".join(top_words))

In [11]:
sub = [lda, cv, nmf, tfidf]

In [4]:
import pickle
pickle.dump(ts, open( "save.p", "wb" ))

In [5]:
import pickle
a = pickle.load(open( "save.p", "rb" ))

In [8]:
a.models['2016-11-09']

[LatentDirichletAllocation(n_components=5, random_state=42),
 CountVectorizer(tokenizer=<function TopicSeries.twitter_tokenizer at 0x7ffe7bc88440>),
 NMF(n_components=5, random_state=5),
 TfidfVectorizer(tokenizer=<function TopicSeries.twitter_tokenizer at 0x7ffe7bc88440>)]

In [12]:
display_components(a.models['2016-05-08'][2], a.models['2016-05-08'][3].get_feature_names(), top_display=20)

Topic 0:
day happy mother mothers mom moms love #mothersday great thank wife nice time celebrating appreciation important today life enjoy world
Topic 1:
trump donald says clinton hillary gop party women republican ryan going president win taxes paul nominee wealthy better candidate said
Topic 2:
lol yep seen look actual idiot irl pic pro true picks trying fireworks nails choice texas nice huh answer haha
Topic 3:
good luck morning point love story man pretty night new nt book true time years listen wo saudi change add
Topic 4:
like know thanks think people yes new look right sure going time saudi oh great actually minister china oil looks
